In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 512kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.58MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 12.7MB/s]


In [2]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_layers = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_layers(x)
        return logits

model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [3]:
epochs = 10

for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {e+1} - Training loss: {running_loss/len(train_loader)}")


Epoch 1 - Training loss: 0.38811432246936917
Epoch 2 - Training loss: 0.17267492605544038
Epoch 3 - Training loss: 0.12580594188992433
Epoch 4 - Training loss: 0.10138842972841408
Epoch 5 - Training loss: 0.08505619049413816
Epoch 6 - Training loss: 0.07655040875250009
Epoch 7 - Training loss: 0.0656833630933889
Epoch 8 - Training loss: 0.060084570065212214
Epoch 9 - Training loss: 0.052902077071627834
Epoch 10 - Training loss: 0.04979984657709866


In [4]:
correct = 0
total = 0


with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)


        _, predicted = torch.max(outputs.data, 1)


        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10,000 test images: {accuracy:.2f}%')

Accuracy of the network on the 10,000 test images: 97.22%
